In [ ]:
# add proprietary modules into the path
#from pathlib import Path
#sys.path.append(str(Path.home().joinpath("AI4ArcticSeaIceChallenge/")))

# -- Built-in modules -- #
import gc
import os
import sys

# -- Environmental variables -- #
os.environ['AI4ARCTIC_DATA'] = '/Volumes/T7/21316608'  # Fill in directory for data location.
os.environ['AI4ARCTIC_ENV'] = '/Users/mepercuter/Desktop/AI4ArcticSeaIceChallenge-main/'  # Fill in directory for environment with Ai4Arctic get-started package. 


In [ ]:
# -- Third-part modules -- #
import json
import matplotlib.pyplot as plt
import numpy as np
import torch
import xarray as xr
from tqdm.notebook import tqdm  # Progress bar
import math

# --Proprietary modules -- #
from functions import chart_cbar, r2_metric, f1_metric, compute_metrics  # Functions to calculate metrics and show the relevant chart colorbar.
from loaders import AI4ArcticChallengeDataset, AI4ArcticChallengeTestDataset, get_variable_options  # Custom dataloaders for regular training and validation.
from AutoIceTransUNet import UNetViT #from unettrans import UNet  # Convolutional Neural Network model
from utils import CHARTS, SIC_LOOKUP, SOD_LOOKUP, FLOE_LOOKUP, SCENE_VARIABLES, colour_str

#Define crop size
crop_size = 512

train_options = {
    # -- Training options -- #
    #'path_to_processed_data': os.environ['AI4ARCTIC_DATA'],
    'path_to_processed_data': os.environ['AI4ARCTIC_DATA'],  # Replace with data directory path.
    'path_to_env': os.environ['AI4ARCTIC_ENV'],  # Replace with environmment directory path.
    'lr': 0.0001,  # Optimizer learning rate.
    'epochs': 120,  # Number of epochs before training stop.
    'epoch_len': 100,  # Number of batches for each epoch.
    'patch_size': int(crop_size),  # Size of patches sampled. Used for both Width and Height.
    'batch_size': 24,  # Number of patches for each batch.
    'loader_upsampling': 'nearest',  # How to upscale low resolution variables to high resolution.
    
    # -- Data prepraration lookups and metrics.
    'train_variables': SCENE_VARIABLES,  # Contains the relevant variables in the scenes.
    'charts': CHARTS,  # Charts to train on.
    'n_classes': {  # number of total classes in the reference charts, including the mask.
        'SIC': SIC_LOOKUP['n_classes'],
        'SOD': SOD_LOOKUP['n_classes'],
        'FLOE': FLOE_LOOKUP['n_classes']
    },
    
    'pixel_spacing': 80,  # SAR pixel spacing. 80 for the ready-to-train AI4Arctic Challenge dataset.
    'train_fill_value': 0,  # Mask value for SAR training data.
    'class_fill_values': {  # Mask value for class/reference data.
        'SIC': SIC_LOOKUP['mask'],
        'SOD': SOD_LOOKUP['mask'],
        'FLOE': FLOE_LOOKUP['mask'],
    },
    
    # -- Validation options -- #
    'chart_metric': {  # Metric functions for each ice parameter and the associated weight.
        'SIC': {
            'func': r2_metric,
            'weight': 2,
        },
        'SOD': {
            'func': f1_metric,
            'weight': 2,
        },
        'FLOE': {
            'func': f1_metric,
            'weight': 1,
        },
    },
    'num_val_scenes': 30,  # Number of scenes randomly sampled from train_list to use in validation.
    
    # -- GPU/cuda options -- #
    'gpu_id': 0,  # Index of GPU. In case of multiple GPUs.
    'num_workers': 6,  # Number of parallel processes to fetch data.
    'num_workers_val': 1,  # Number of parallel processes during validation.
    
    # -- U-Net Options -- #
    'unet_conv_filters': [16, 32, 64, 128],  # Number of filters in the U-Net.
    'conv_kernel_size': (3, 3),  # Size of convolutional kernels.
    'conv_stride_rate': (1, 1),  # Stride rate of convolutional kernels.
    'conv_dilation_rate': (1, 1),  # Dilation rate of convolutional kernels.
    'conv_padding': (1, 1),  # Number of padded pixels in convolutional layers.
    'conv_padding_style': 'zeros',  # Style of padding.
}
# Get options for variables, amsrenv grid, cropping and upsampling.
get_variable_options = get_variable_options(train_options)
# To be used in test_upload.
%store train_options  

# Load training list.
#with open('AI4ArcticSeaIceChallenge/datalists/dataset.json') as file:
#    train_options['train_list'] = json.loads(file.read())
with open(train_options['path_to_env'] + 'datalists/dataset.json') as file:
    train_options['train_list'] = json.loads(file.read())
# Convert the original scene names to the preprocessed names.
train_options['train_list'] = [file[17:32] + '_' + file[77:80] + '_prep.nc' for file in train_options['train_list']]
# Select a random number of validation scenes with the same seed. Feel free to change the seed.et
np.random.seed(0)
train_options['validate_list'] = np.random.choice(np.array(train_options['train_list']), size=train_options['num_val_scenes'], replace=False)
# Remove the validation scenes from the train list.
train_options['train_list'] = [scene for scene in train_options['train_list'] if scene not in train_options['validate_list']]
train_options['test_list'] = np.random.choice(np.array(train_options['train_list']), size=20, replace=False)
train_options['test_list1'] = np.random.choice(np.array(train_options['test_list']), size=10, replace=False)
# Remove the validation scenes from the train list.
train_options['train_list'] = [scene for scene in train_options['train_list'] if scene not in train_options['test_list']]
# Remove the validation scenes from the train list.
train_options['test_list'] = [scene for scene in train_options['test_list'] if scene not in train_options['test_list1']]
print(len(train_options['train_list']))
print(len(train_options['validate_list']))
print(len(train_options['test_list']))
print(len(train_options['test_list1']))
print(len(train_options['charts']))
print('Options initialised')

print('Setup ready')


In [ ]:
# Get GPU resources.
if torch.cuda.is_available():
    print(colour_str('GPU available!', 'green'))
    print('Total number of available devices: ', colour_str(torch.cuda.device_count(), 'orange'))
    device = torch.device(f"cuda:{train_options['gpu_id']}")

else:
    print(colour_str('GPU not available.', 'red'))
    device = torch.device('cpu')

# Custom dataset and dataloader.
dataset = AI4ArcticChallengeDataset(files=train_options['train_list'], options=train_options)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=None, shuffle=True, num_workers=train_options['num_workers'], pin_memory=True)
# - Setup of the validation dataset/dataloader. The same is used for model testing in 'test_upload.ipynb'.
dataset_val = AI4ArcticChallengeTestDataset(options=train_options, files=train_options['validate_list'])
dataloader_val = torch.utils.data.DataLoader(dataset_val, batch_size=None, num_workers=train_options['num_workers_val'], shuffle=False)
# - Setup of the test dataset/dataloader. The same is used for model testing in 'test_upload.ipynb'.
dataset_test = AI4ArcticChallengeTestDataset(options=train_options, files=train_options['test_list'])
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=None, num_workers=train_options['num_workers_val'], shuffle=False)
# - Setup of the test dataset/dataloader. The same is used for model testing in 'test_upload.ipynb'.
dataset_test1 = AI4ArcticChallengeTestDataset(options=train_options, files=train_options['test_list1'])
dataloader_test1 = torch.utils.data.DataLoader(dataset_test1, batch_size=None, num_workers=train_options['num_workers_val'], shuffle=False)

print('GPU and data setup complete.')

In [ ]:
# Setup U-Net model, adam optimizer, loss function and dataloader.
#UNetViT(in_channels=24, num_classes=64).to(device) #
model = UNetViT(in_channels=24, num_classes=16).to(device) #net = UNet(options=train_options).to(device)
optimizer = torch.optim.Adam(list(model.parameters()), lr=train_options['lr'])
torch.backends.cudnn.benchmark = True  # Selects the kernel with the best performance for the GPU and given input size.

# Loss functions to use for each sea ice parameter.
# The ignore_index argument discounts the masked values, ensuring that the model is not using these pixels to train on.
# It is equivalent to multiplying the loss of the relevant masked pixel with 0.
loss_functions = {chart: torch.nn.CrossEntropyLoss(ignore_index=train_options['class_fill_values'][chart]) \
                                                   for chart in train_options['charts']}
#model.load_state_dict(torch.load('best_model', map_location=torch.device('cpu'))['model_state_dict'])
print('Model setup complete')

In [ ]:
#model.load_state_dict(torch.load('best_model', map_location=torch.device('cpu'))['model_state_dict'])
Test_folder = 'testtest'

In [ ]:
print('Testing 1.')

os.makedirs(f"{Test_folder}", exist_ok=True)
# - Stores the output and the reference pixels to calculate the scores after inference on all the scenes.
outputs_flat = {chart: np.array([]) for chart in train_options['charts']}
inf_ys_flat = {chart: np.array([]) for chart in train_options['charts']}

model.eval()  # Set network to evaluation mode.
i=0
# - Loops though scenes in queue.
for inf_x, inf_y, masks, name in tqdm(iterable=asid_loader, total=len(train_options['test_list']), colour='green', position=0):
    scene_name = name[:19]  # Removes the _prep.nc from the name.
    torch.cuda.empty_cache()
    inf_x = inf_x.to(device, non_blocking=True)

    # - Ensures that no gradients are calculated, which otherwise take up a lot of space on the GPU.
    with torch.no_grad(), torch.cuda.amp.autocast():
        output = model(inf_x)

    # - Final output layer, and storing of non masked pixels.
    for chart in train_options['charts']:
        output[chart] = torch.argmax(output[chart], dim=1).squeeze().cpu().numpy()
        outputs_flat[chart] = np.append(outputs_flat[chart], output[chart][~masks[chart]])
        inf_y[chart] = inf_y[chart].squeeze()
        inf_ys_flat[chart] = np.append(inf_ys_flat[chart], inf_y[chart][~masks[chart]].numpy())
        inf_y[chart] = inf_y[chart].cpu().numpy()
        
    inf_x = inf_x.squeeze()
    inf_x = inf_x.cpu().numpy()
    inf_x1 = inf_x
    
    del inf_x
        
    # - Show the scene inference.
    fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(10, 10))
    for idx, chart in enumerate(train_options['charts']):
        ax = axs[0,idx]
        output[chart] = output[chart].astype(float)
        output[chart][masks[chart]] = np.nan
        ax.imshow(output[chart], vmin=0, vmax=train_options['n_classes'][chart] - 2, cmap='jet', interpolation='nearest')
        ax.set_xticks([])
        ax.set_yticks([])
        chart_cbar(ax=ax, n_classes=train_options['n_classes'][chart], chart=chart, cmap='jet')
        ax1 = axs[1,idx]
        inf_y[chart] = inf_y[chart].astype(float)
        inf_y[chart][masks[chart]] = np.nan
        ax1.imshow(inf_y[chart], vmin=0, vmax=train_options['n_classes'][chart] - 2, cmap='jet', interpolation='nearest')
        ax1.set_xticks([])
        ax1.set_yticks([])
        chart_cbar(ax=ax1, n_classes=train_options['n_classes'][chart], chart=chart, cmap='jet')
    ax2 = axs[2,0]
    inf_x1[0] = inf_x1[0].astype(float)
    ax2.imshow(inf_x1[0], cmap='gray')
    ax2.set_xticks([])
    ax2.set_yticks([])
    ax3 = axs[2,1]
    inf_x1[1] = inf_x1[1].astype(float)
    ax3.imshow(inf_x1[1], cmap='gray')
    ax3.set_xticks([])
    ax3.set_yticks([])
    ax4 = axs[2,2]
    inf_x1[3] = inf_x1[3].astype(float)
    ax4.imshow(inf_x1[3], cmap='gray')
    ax4.set_xticks([])
    ax4.set_yticks([])
    
    plt.suptitle(f"SIC, SoD and FLOE predictions. Scene: {scene_name}", y=1)
    plt.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0.5, hspace=-0)
    fig.savefig(f"{Test_folder}/{scene_name}.png", format='png', dpi=128, bbox_inches="tight")
    plt.close('all')

    del inf_x1, inf_y, masks, output, fig, axs, ax, ax1, ax2, ax3, ax4  # Free memory.

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd

# Build confusion matrix
cf_matrix_SIC = confusion_matrix(inf_ys_flat['SIC'], outputs_flat['SIC'])

SIC_GROUPS = ('0','10','20','30','40','50','60','70','80','90','100')
SOD_GROUPS = ('Open water','New Ice','Young ice','Thin FYI','Thick FYI','Old ice')
FLOE_GROUPS = ('Open water','Cake Ice','Small floe','Medium floe','Big floe','Vast floe','Bergs')

cf_matrix_SOD = confusion_matrix(inf_ys_flat['SOD'], outputs_flat['SOD'])


cf_matrix_FLOE = confusion_matrix(inf_ys_flat['FLOE'], outputs_flat['FLOE'])

# - Compute the relevant scores.
print('Calculating Metrics for Testing')
combined_score, scores = compute_metrics(true=inf_ys_flat, pred=outputs_flat, charts=train_options['charts'],
                                         metrics=train_options['chart_metric'])

print('Metrics Calculated for Testing')

print("")
for chart in train_options['charts']:
    print(f"{chart} {train_options['chart_metric'][chart]['func'].__name__}: {scores[chart]}%")
print(f"Combined score: {combined_score}%")
del inf_ys_flat, outputs_flat  # Free memory.

In [ ]:
print('Testing 2.')
# - Stores the output and the reference pixels to calculate the scores after inference on all the scenes.
outputs_flat = {chart: np.array([]) for chart in train_options['charts']}
inf_ys_flat = {chart: np.array([]) for chart in train_options['charts']}

model.eval()  # Set network to evaluation mode.
i=0
# - Loops though scenes in queue.
for inf_x, inf_y, masks, name in tqdm(iterable=asid_loader1, total=len(train_options['test_list1']), colour='green', position=0):
    scene_name = name[:19]  # Removes the _prep.nc from the name.
    torch.cuda.empty_cache()
    inf_x = inf_x.to(device, non_blocking=True)

    # - Ensures that no gradients are calculated, which otherwise take up a lot of space on the GPU.
    with torch.no_grad(), torch.cuda.amp.autocast():
        output = model(inf_x)

    # - Final output layer, and storing of non masked pixels.
    for chart in train_options['charts']:
        output[chart] = torch.argmax(output[chart], dim=1).squeeze().cpu().numpy()
        outputs_flat[chart] = np.append(outputs_flat[chart], output[chart][~masks[chart]])
        inf_y[chart] = inf_y[chart].squeeze()
        inf_ys_flat[chart] = np.append(inf_ys_flat[chart], inf_y[chart][~masks[chart]].numpy())
        inf_y[chart] = inf_y[chart].cpu().numpy()
        
    inf_x = inf_x.squeeze()
    inf_x = inf_x.cpu().numpy()
    inf_x1 = inf_x
    
    del inf_x
        
    # - Show the scene inference.
    fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(10, 10))
    for idx, chart in enumerate(train_options['charts']):
        ax = axs[0,idx]
        output[chart] = output[chart].astype(float)
        output[chart][masks[chart]] = np.nan
        ax.imshow(output[chart], vmin=0, vmax=train_options['n_classes'][chart] - 2, cmap='jet', interpolation='nearest')
        ax.set_xticks([])
        ax.set_yticks([])
        chart_cbar(ax=ax, n_classes=train_options['n_classes'][chart], chart=chart, cmap='jet')
        ax1 = axs[1,idx]
        inf_y[chart] = inf_y[chart].astype(float)
        inf_y[chart][masks[chart]] = np.nan
        ax1.imshow(inf_y[chart], vmin=0, vmax=train_options['n_classes'][chart] - 2, cmap='jet', interpolation='nearest')
        ax1.set_xticks([])
        ax1.set_yticks([])
        chart_cbar(ax=ax1, n_classes=train_options['n_classes'][chart], chart=chart, cmap='jet')
    ax2 = axs[2,0]
    inf_x1[0] = inf_x1[0].astype(float)
    ax2.imshow(inf_x1[0], cmap='gray')
    ax2.set_xticks([])
    ax2.set_yticks([])
    ax3 = axs[2,1]
    inf_x1[1] = inf_x1[1].astype(float)
    ax3.imshow(inf_x1[1], cmap='gray')
    ax3.set_xticks([])
    ax3.set_yticks([])
    ax4 = axs[2,2]
    inf_x1[3] = inf_x1[3].astype(float)
    ax4.imshow(inf_x1[3], cmap='gray')
    ax4.set_xticks([])
    ax4.set_yticks([])
    
    plt.suptitle(f"SIC, SoD and FLOE predictions. Scene: {scene_name}", y=1)
    plt.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0.5, hspace=-0)
    fig.savefig(f"{Test_folder}/{scene_name}.png", format='png', dpi=128, bbox_inches="tight")
    plt.close('all')

    del inf_x1, inf_y, masks, output, fig, axs, ax, ax1, ax2, ax3, ax4  # Free memory.

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd

# Build confusion matrix
cf_matrix_SIC1 = confusion_matrix(inf_ys_flat['SIC'], outputs_flat['SIC'])

SIC_GROUPS = ('0','10','20','30','40','50','60','70','80','90','100')
SOD_GROUPS = ('Open water','New Ice','Young ice','Thin FYI','Thick FYI','Old ice')
FLOE_GROUPS = ('Open water','Cake Ice','Small floe','Medium floe','Big floe','Vast floe','Bergs')

# Build confusion matrix
df_cm_SIC = pd.DataFrame((cf_matrix_SIC+cf_matrix_SIC1) / np.sum((cf_matrix_SIC+cf_matrix_SIC1), axis=1)[:, None], index = [i for i in SIC_GROUPS],
                     columns = [i for i in SIC_GROUPS])
plt.figure(figsize = (12,7))
sn.heatmap(df_cm_SIC, annot=True, fmt='.2%', cmap='Blues')
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.savefig(f'{Test_folder}/output1_SIC.png')


cf_matrix_SOD1 = confusion_matrix(inf_ys_flat['SOD'], outputs_flat['SOD'])
df_cm_SOD = pd.DataFrame((cf_matrix_SOD+cf_matrix_SOD1) / np.sum((cf_matrix_SOD+cf_matrix_SOD1), axis=1)[:, None], index = [i for i in SOD_GROUPS],
                     columns = [i for i in SOD_GROUPS])
plt.figure(figsize = (12,7))
sn.heatmap(df_cm_SOD, annot=True, fmt='.2%', cmap='Blues')
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.savefig(f'{Test_folder}/output1_SOD.png')


cf_matrix_FLOE1 = confusion_matrix(inf_ys_flat['FLOE'], outputs_flat['FLOE'])
df_cm_FLOE = pd.DataFrame((cf_matrix_FLOE+cf_matrix_FLOE1) / np.sum((cf_matrix_FLOE+cf_matrix_FLOE1), axis=1)[:, None], index = [i for i in FLOE_GROUPS],
                     columns = [i for i in FLOE_GROUPS])
plt.figure(figsize = (12,7))
sn.heatmap(df_cm_FLOE, annot=True, fmt='.2%', cmap='Blues')
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.savefig(f'{Test_folder}/output_FLOE.png')

# - Compute the relevant scores.
print('Calculating Metrics for Testing')
combined_score, scores = compute_metrics(true=inf_ys_flat, pred=outputs_flat, charts=train_options['charts'],
                                         metrics=train_options['chart_metric'])

print('Metrics Calculated for Testing')

print("")
for chart in train_options['charts']:
    print(f"{chart} {train_options['chart_metric'][chart]['func'].__name__}: {scores[chart]}%")
print(f"Combined score: {combined_score}%")
del inf_ys_flat, outputs_flat  # Free memory.

In [ ]:
best_combined_score = 0  # Best weighted model score.
learn_rate = train_options['lr']#0.0001#
# -- Training Loop -- #
for epoch in tqdm(iterable=range(train_options['epochs']), position=0):
    
    optimizer = torch.optim.Adam(list(model.parameters()), lr=learn_rate)
    gc.collect()  # Collect garbage to free memory.
    loss_sum = torch.tensor([0.])  # To sum the batch losses during the epoch.
    model.train()  # Set network to evaluation mode.

    # Loops though batches in queue.
    for i, (batch_x, batch_y) in enumerate(tqdm(iterable=dataloader, total=train_options['epoch_len'], colour='red', position=0)):
        torch.cuda.empty_cache()  # Empties the GPU cache freeing up memory.
        loss_batch = 0  # Reset from previous batch.
        
        # - Transfer to device.
        batch_x = batch_x.to(device, non_blocking=True)

        # - Mixed precision training. (Saving memory)
        with torch.cuda.amp.autocast():
            output = model(batch_x)
            # - Calculate loss.
            for chart in train_options['charts']:
                loss_batch += loss_functions[chart](input=output[chart], target=batch_y[chart].to(device))

        # - Reset gradients from previous pass.
        optimizer.zero_grad()

        # - Backward pass.
        loss_batch.backward()

        # - Optimizer step
        optimizer.step()

        # - Add batch loss.
        loss_sum += loss_batch.detach().item()

        # - Average loss for displaying
        loss_epoch = torch.true_divide(loss_sum, i + 1).detach().item()
        print('\rMean training loss: ' + f'{loss_epoch:.3f}', end='\r')
        del output, batch_x, batch_y # Free memory.
    del loss_sum
    
    # -- Validation Loop -- #
    loss_batch = loss_batch.detach().item()  # For printing after the validation loop.
    
    # - Stores the output and the reference pixels to calculate the scores after inference on all the scenes.
    outputs_flat = {chart: np.array([]) for chart in train_options['charts']}
    inf_ys_flat = {chart: np.array([]) for chart in train_options['charts']}

    model.eval()  # Set network to evaluation mode.
    # - Loops though scenes in queue.
    loss_val = 0
    for inf_x, inf_y, masks, name in tqdm(iterable=dataloader_val, total=len(train_options['validate_list']), colour='green', position=0):
        scene_name = name[:19]  # Removes the _prep.nc from the name.
        torch.cuda.empty_cache()
        
        # - Ensures that no gradients are calculated, which otherwise take up a lot of space on the GPU.
        with torch.no_grad(), torch.cuda.amp.autocast():
            inf_x = inf_x.to(device, non_blocking=True)
            output = model(inf_x)
    
        # - Final output layer, and storing of non masked pixels.
        for chart in train_options['charts']:
            inf_y[chart] = inf_y[chart].unsqueeze(0)
            loss_val += loss_functions[chart](input=output[chart], target=inf_y[chart].long().to(device))
            output[chart] = torch.argmax(output[chart], dim=1).squeeze().cpu().numpy()
            inf_y[chart] = inf_y[chart].squeeze()
            outputs_flat[chart] = np.append(outputs_flat[chart], output[chart][~masks[chart]])
            inf_ys_flat[chart] = np.append(inf_ys_flat[chart], inf_y[chart][~masks[chart]].numpy())
            
        del inf_x, inf_y, masks, output  # Free memory.

    # - Compute the relevant scores.
    print('Calculating Metrics')
    combined_score, scores = compute_metrics(true=inf_ys_flat, pred=outputs_flat, charts=train_options['charts'],
                                             metrics=train_options['chart_metric'])
    
    print('Metrics Calculated')

    print("")
    print(f"Final batch loss: {loss_batch:.3f}")
    print(f"Epoch {epoch} score:")
    print(f"Validation loss: {loss_val/(train_options['num_val_scenes']):.3f}")
    for chart in train_options['charts']:
        print(f"{chart} {train_options['chart_metric'][chart]['func'].__name__}: {scores[chart]}%")
    print(f"Combined score: {combined_score}%")

    # If the scores is better than the previous epoch, then save the model and rename the image to best_validation.
    if combined_score > best_combined_score:
        best_combined_score = combined_score
        torch.save(obj={'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'epoch': epoch},
                        f='best_model')
    del inf_ys_flat, outputs_flat  # Free memory.
